In [1]:
import tensorflow as tf

In [2]:
import os

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
import read_tree as rt

import sys


Welcome to JupyROOT 6.13/03


In [6]:
import json

In [7]:
%matplotlib inline

In [8]:
"""
ten nowy bedzie uproszczony
__init__(nazwa_folderu,tryb,co_ile_flush_file=10)
    inputs:
            nazwa_folderu np "pierwszy_dataset" tam bedzie pisac/stamtad bedzie sczytywac. 
            tryb np 'r' lub 'w' i oznacza czy czytac chcesz czy pisac
            co_ile_flush_file to znaczy jak czesto ma oprozniac swoj buffer, liczy sie tylko
                    gdy tryb=='w', nie wiem ile ma wynosic,wiec jak wiesz to smialo ustaw

write(legs,jets,global_params,properties,l)
        dostepna tylko jak tryb to jest 'w'
            legs,jets,global_params,properties jak w wyjsciu klasy read_tree, tylko "dla jednego przypadku"
                wiec jest tak
                        zakladam, ze legs to jest lista o shapie (?,4) wypelniona floatami
                        jets dokladnie tak samo
                        global_params to jest w postaci {nazwa:liczba, ...}
                        properties tak samo
                        l to label jest intem rownym to 0 lub 1, gdzie 1 oznacza, ze to jest raczej bardziej ciekawy przypadek
                         a 0 to taki bardziej tlo. to jest int 

close()
            dostepna tylko dla tryb=='w'
            zamyka bezpiecznie nasz plik
read()
            dostepna tylko dla tryb=='r'
            wyrzuci z siebie tensorflowowy dataset gotowy do uczenia
                w przyszlosci read moze przeprowadzac dodawanie nowych featcherkow,ale na razie
                tego nie robi. W sumie jak mamy dataset to mozemy tez te featcherki nowe dorobic poxniej.
                zobaczymy jak to bedzie.
            dataset wyglada tak, ze pojedynczy przypadek to jest (slownik_featurow,label)
                gdzie label to bedzie 0 lub 1
                zas slownik featerow to jest tak, ze sa klucze stringow a wartosci to jest
                1d arraye ktore sa intami lub floatami 
                to znaczy, ze zwraca to samo co ta stara moja klasa czytajaca
types()
            uzywamy takiego czegos tylko dla type=='r'
            zwraca slownik typow naszego datasetu
            






przyjmuje w konstruktorze nazwe folderu gddzie bedzie pisac (utworzy go jak go nie ma)
(przyjmujemy zasade jeden zbior danych w jednym folderze, a nazwa folderu jest informatywna)

oraz tryb 'r' do czytania 'w' do pisania

copy buffer
self.iotf_stary=Io_tf_binary_general.Io_tf_binary_stary(
            self.nazwa_folderu+"/dane",slownik,tryb,co_ile_flush_file)
            
w funcji  write w argumentach zakladam
#zakladam, ze legs to jest lista o shapie (?,4) wypelniona floatami
#jets dokladnie tak samo
#global_params to jest w postaci {nazwa:liczba, ...}
#properties tak samo
#l jest intem rownym to 0 lub 1, gdzie 1 oznacza, ze to jest raczej bardziej ciekawy przypadek
# a 0 to taki bardziej tlo

jest tez funkcja close ktora odcina nas od pliku zapisywanego

funkcja read na razie nie bierze argumentow, ale w przyszlosci bedzie robic feature engineering
to znaczy dorobi jakies z tych podstawowych typow
"""


class Io_tf_binary_general:
    def __init__(self,nazwa_folderu,tryb,co_ile_flush_file=10):
        
        self.nazwa_folderu=nazwa_folderu
        self.tryb=tryb
        self.co_ile=co_ile_flush_file
        self.nowopowstala=True
        
    
    def zapisz_json(co,gdzie):
        f=open(gdzie,'w')
        f.write(json.dumps(co))
    def wczytaj_json(skad):
        f=open(skad,'r')
        return json.loads(f.read())
    
    def zrob_slownik_typow(legs,jets,global_params,properties):
        #zakladam, ze legs to jest lista o shapie (?,4) wypelniona floatami
        #jets dokladnie tak samo
        #global_params to jest w postaci {nazwa:liczba, ...}
        wyrzut={}
        legs=np.array(legs)
        jets=np.array(jets)
        n_legs=legs.shape[0]
        n_jets=jets.shape[0]
        def czy_int(x):
            type(x)==int or type(x)==int ==numpy.int64

        for i in range(n_legs):
            wyrzut["leg_"+str(i)+"_momentum"]=(4,'f')
        for i in range(n_jets):
            wyrzut["jet_"+str(i)+"_momentum"]=(4,'f')
        def dorob(wyrzut,global_params):
            for param_key in global_params.keys():
                if np.issubdtype(type(global_params[param_key]), np.integer):
                    wyrzut[param_key]=(1,'i')
                else:
                    wyrzut[param_key]=(1,'f')
            return wyrzut
        wyrzut=dorob(wyrzut,global_params)
        wyrzut=dorob(wyrzut,properties)
        return wyrzut
    
    def zrob_sensowna_forme(legs,jets,global_params,properties,l):
        #l jest intem i to 0 lub 1
        legs=np.array(legs)
        jets=np.array(jets)
        n_legs=legs.shape[0]
        n_jets=jets.shape[0]

        f={}


        for i in range(n_legs):
            f["leg_"+str(i)+"_momentum"]=legs[i,:]
        for i in range(n_jets):
            f["jet_"+str(i)+"_momentum"]=jets[i,:]

        for param_key in global_params.keys():
            f[param_key]=[global_params[param_key]]
        for k in properties:
            f[k]=[properties[k]]
        return f,l
    
    def write(self,legs,jets,global_params,properties,l):
        #zakladam, ze legs to jest lista o shapie (?,4) wypelniona floatami
        #jets dokladnie tak samo
        #global_params to jest w postaci {nazwa:liczba, ...}
        #l jest intem i to 0 lub 1
        assert self.tryb=='w'
        if self.nowopowstala==True:
            os.system("mkdir "+self.nazwa_folderu)
            self.nowopowstala=False
            slownik= Io_tf_binary_general.zrob_slownik_typow(legs,jets,global_params,properties)
            Io_tf_binary_general.zapisz_json(slownik,self.nazwa_folderu+"/metadata")
            self.stary_io=Io_tf_binary_general.Io_tf_binary_stary(
                self.nazwa_folderu+"/dane",slownik,self.tryb,self.co_ile)
        f,l=Io_tf_binary_general.zrob_sensowna_forme(legs,jets,global_params,properties,l)
        self.stary_io.wpisz(f,l)
    def close(self):
        assert self.tryb=='w'
        if not self.nowopowstala:
            self.stary_io.close()
    def read(self):
        assert self.tryb=='r'
        slownik=Io_tf_binary_general.wczytaj_json(self.nazwa_folderu+"/metadata")
        stary_io=Io_tf_binary_general.Io_tf_binary_stary(
                self.nazwa_folderu+"/dane",slownik,self.tryb,self.co_ile)
        return stary_io.wczytaj_dataset()
    def types(self):
        assert self.tryb=='r'
        return Io_tf_binary_general.wczytaj_json(self.nazwa_folderu+"/metadata")
    
    #jakby ktos kopiowal to to idzie dalej
    #==========================================================    
        """
    do creatora dajemy sobie nazwe pliku (sciezke) oraz slownik typu {'czterowektor': (4,'f'),'intowa_wlasnosc': (1,'i'),
    ...}
    to znaczy nazwe, ile to jest liczb, jakiego typu. Obsluguje na razie jedynie 'f' oraz 'i' to 
    jest float oraz int
    nie mozna uzyc jako klucz 'label', bo to jest wykorzystywana nazwa.


    funkcja wpisz bierze jako argument jeden przypadek  cos typu 
    ({"czterowektor":[1.,2.,3.,4.], ...},1) gdzie 1 jest labelem, label jest intem.
    . oczywiście klucze slownika zgadzają się 
    z kluczami ze slownika ktorego uzylismy do kreatora.

    dataset wczytany metoda wczytaj dataset jest juz w postaci wygodnej dla mnie to znaczy 
    slownik feature, label
    """
    class Io_tf_binary_stary:
        def __init__(self,nazwa_pliku,slownik,tryb,co_ile_flush_file=10):
            #tryb to 'w' dla write oraz 'r' dla read
            self.co_ile=co_ile_flush_file
            self.plik=nazwa_pliku
            self.typy=slownik
            for k in self.typy.keys():
                assert self.typy[k][1] in ['f','i']
                assert self.typy[k][0]>0 
                assert np.issubdtype(type(self.typy[k][0]), np.integer)
            if tryb=='w':
                self.writer= tf.python_io.TFRecordWriter(self.plik)
            self.liczba_wrzuconych=0

            #self.cos=Io_tf_binary.wrap_int64([5])

        def close(self):
            self.writer.close()


        """
        To moze sobie czytac ktos kto chce zmieniac wnetrznosci tej klasy
        Nie polecam

        Teraz ta funkcja wpisz jest ważna. ona bierze po jednym przypadku testowym, 
        ( to jest ta petla for i in range()) i go zapisuje. trzeba zwracac 
        uwage na to jakiego typu sa zapisywane rzeczy. mozna oczywiscie zrobic slownik
        data dluzszym, jesli to w jakis sposob ulatwi nam myslenie o naszych danych. 
        Bo te nasze dane to bedzie slownik list, w ktorych to listach rzeczy maja 
        juz taki sam typ, a klucze to beda jakies opisowe nazwy.
        np 

        data={
        'czteroped_lewej_nogi_czy_cos': wrap_float64(cztero), # gdzie cztero to jest tensor floatow o shape (4,)
        # reszta rzeczy

        }

        Jak byscie chcieli jako wartosci miec stringi to musicie pomyslec jak zrobic wrapy dla stringow. oczywiscie
        nie znajdziecie zadnej dokumentacji.

        UWAGA 
        w tym slowniku data musi byc to co klasyfikujemy oznaczone przy pomocy 'label' bo inaczej sie  wywali program.





        """


        def wpisz(self,features,label):
            """tworzy ten nasz dataset w pliku out_path 
            tu format jest taki jak byl na poczatku to znaczy taki slownik"""
            f=features
            l=label
            def wrap_int64(value):
                """lista intow musi wlesc"""
                return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
            def wrap_float64(value):
                """lista floatow musi wlesc"""
                return tf.train.Feature(float_list=tf.train.FloatList(value=value))

            #f,l=kolko_w_kolku() #mozna zmienic jak sie podoba
            def data_slownik(f,l):
                wyrzut={}
                for k in self.typy.keys():
                    if self.typy[k][1]=='f':
                        wyrzut[k]=wrap_float64(np.array(f[k]).reshape((-1,)))
                    else:
                        wyrzut[k]=wrap_int64(np.array(f[k]).reshape((-1,)))
                wyrzut['label']=wrap_int64([l])
                return wyrzut



            #feature=f[i]
            #label=l[i]
            #data = {
             #    'feature': wrap_float64(feature),
            #  'label': wrap_int64([label])
               #     }
            data=data_slownik(f,l)
            # Wrap the data as TensorFlow Features.
            feature = tf.train.Features(feature=data)

            # Wrap again as a TensorFlow Example.
            example = tf.train.Example(features=feature)

            # Serialize the data.
            serialized = example.SerializeToString()

            # Write the serialized data to the TFRecords file.
            self.writer.write(serialized)
            self.liczba_wrzuconych+=1
            if self.liczba_wrzuconych%self.co_ile==0:
                self.writer.flush()










        def wczytaj_dataset(self):

            def zeslownikoj(x):
                keys=list(x.keys())
                f={}
                for k in keys:
                    if not k=='label':
                        f[k]=x[k]
                return f,x['label']
            def features_generoj():
                wyrzut={}
                for k in self.typy.keys():
                    if self.typy[k][1]=='f':
                        wyrzut[k]=tf.FixedLenFeature([self.typy[k][0]], tf.float32)
                    else:
                        wyrzut[k]=tf.FixedLenFeature([self.typy[k][0]], tf.int64)
                wyrzut['label']=tf.FixedLenFeature([], tf.int64)
                return wyrzut

            def parse(serialized):

                # Define a dict with the data-names and types we expect to
                # find in the TFRecords file.
                # It is a bit awkward that this needs to be specified again,
                # because it could have been written in the header of the
                # TFRecords file instead.
                """
                features = \
                    {
                        'dwuwektor': tf.FixedLenFeature([2], tf.float32),#z jakiegos powodu to jest float32, nie wiem czemu
                        'label': tf.FixedLenFeature([], tf.int64)
                    }
                """
                features=features_generoj()
                print(features)

                # Parse the serialized data so we get a dict with our data.
                parsed_example = tf.parse_single_example(serialized=serialized,
                                                         features=features)


                return zeslownikoj(parsed_example)

            dataset = tf.data.TFRecordDataset(self.plik)
            dataset = dataset.map(parse)
            return dataset
    
    
    
    
    
    
    
    
    
    
    

In [9]:
in_file="data/dummy.root"
out_file="output/example"
tree_path="Summary/tree"
legs, jets, global_params, properties = rt.read_tree(in_file, tree_path)

[ML]	Opening root file for python conversion.
[ML]	Reading data from TTree.
[ML]	Conversion to python successful!


In [10]:
np.array(legs).shape[2]

637

In [11]:
def wybierz(slownik,i):
    wyrzut={}
    for k in slownik.keys():
        wyrzut[k]=slownik[k][i]
    return wyrzut

In [12]:
#poczatek przykladu

In [13]:
pisacz=Io_tf_binary_general("pierwszy",'w')

In [14]:
for i in range(np.array(legs).shape[2]):
    pisacz.write(np.array(legs)[:,:,i],np.array(jets)[:,:,i],wybierz(global_params,i),wybierz(properties,i),0)

mkdir: cannot create directory ‘pierwszy’: File exists


In [15]:
pisacz.close()

In [16]:
czytacz=Io_tf_binary_general("pierwszy",'r')

In [17]:
dataset=czytacz.read()

{'leg_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'leg_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'BJetBetaScore': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsMassTrans': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsPT': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'nJets30': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'visMass': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_charge': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_combreliso': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': FixedLenFeature(shape=[1], dtype=tf.float32, de

In [18]:
czytacz.types()

{'leg_0_momentum': [4, 'f'],
 'leg_1_momentum': [4, 'f'],
 'jet_0_momentum': [4, 'f'],
 'jet_1_momentum': [4, 'f'],
 'BJetBetaScore': [1, 'f'],
 'higgsMassTrans': [1, 'f'],
 'higgsPT': [1, 'f'],
 'nJets30': [1, 'f'],
 'visMass': [1, 'f'],
 'leg_1_charge': [1, 'f'],
 'leg_1_combreliso': [1, 'f'],
 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': [1, 'f'],
 'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': [1, 'f'],
 'leg_2_charge': [1, 'f'],
 'leg_2_decayMode': [1, 'f']}

In [19]:
BATCH_SIZE=2
zbachowany=dataset.shuffle(1000).repeat().batch(BATCH_SIZE)
iterator = zbachowany.make_one_shot_iterator()
para=iterator.get_next()

with tf.Session() as sess:
    for i in range(3):
        print(sess.run(para))

({'BJetBetaScore': array([[-999.],
       [-999.]], dtype=float32), 'higgsMassTrans': array([[101.59774],
       [ 92.35652]], dtype=float32), 'higgsPT': array([[28.415644],
       [40.926834]], dtype=float32), 'jet_0_momentum': array([[ 606.46423  ,   -2.0735188,  -28.639431 , -605.7476   ],
       [   0.       ,    0.       ,    0.       ,    0.       ]],
      dtype=float32), 'jet_1_momentum': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32), 'leg_0_momentum': array([[ 55.597507,  39.645374, -32.87137 ,  20.94729 ],
       [ 36.391502,  29.55554 , -19.151098,  -9.167096]], dtype=float32), 'leg_1_charge': array([[-1.],
       [-1.]], dtype=float32), 'leg_1_combreliso': array([[0.02248299],
       [0.02248375]], dtype=float32), 'leg_1_momentum': array([[ 40.830276  , -34.75293   ,   5.0724854 , -20.82296   ],
       [ 32.664463  ,  24.164114  ,  -0.59195006, -21.960882  ]],
      dtype=float32), 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': array([[1.5285704],
   